### Modules importeren

In [123]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

import os
from loguru import logger # pip install loguru
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

servername = 'LAPTOP-NBUM6TMN' # PAS DIT AAN NAAR JE EIGEN SERVER!!!

# Data inlezen

### Connecties maken

In [124]:
# SQLite Connecties
logger.info('Establishing connection with SQLite databases...')
northwind_sqlite_conn = sqlite3.connect('../data/processed/northwind.sqlite')
adventureworks_sqlite_conn = sqlite3.connect('../data/processed/adventureworks.sqlite')
aenc_sqlite_conn = sqlite3.connect('../data/processed/aenc.sqlite')
logger.success('Connections established!')

# Connect to the SQLite database
conn = sqlite3.connect('../data/processed/adventureworks.sqlite')
cursor = conn.cursor()


2024-05-20 17:48:23.817 | INFO     | __main__:<module>:2 - Establishing connection with SQLite databases...
2024-05-20 17:48:23.819 | SUCCESS  | __main__:<module>:6 - Connections established!


# Dataframes maken

### Person

In [125]:
#Collecting tables
person = pd.read_sql('SELECT * FROM "Person.Person"', adventureworks_sqlite_conn)
businessEntity = pd.read_sql('SELECT * FROM "Person.BusinessEntity"', adventureworks_sqlite_conn)
password = pd.read_sql('SELECT * FROM "Person.Password"', adventureworks_sqlite_conn)
emailAddress = pd.read_sql('SELECT * FROM "Person.EmailAddress"', adventureworks_sqlite_conn)
personPhone = pd.read_sql('SELECT * FROM "Person.PersonPhone"', adventureworks_sqlite_conn)
phoneNumberType = pd.read_sql('SELECT * FROM "Person.PhoneNumberType"', adventureworks_sqlite_conn)
businessEntityContact = pd.read_sql('SELECT * FROM "Person.BusinessEntityContact"', adventureworks_sqlite_conn)
contactType = pd.read_sql('SELECT * FROM "Person.ContactType"', adventureworks_sqlite_conn)

#Merges
person = pd.merge(person, businessEntity, on=['BusinessEntityID'], how='outer', suffixes=('', '_businessEntity'))
person = pd.merge(person, password, on=['BusinessEntityID'], how='outer', suffixes=('', '_password'))
person = pd.merge(person, emailAddress, on=['BusinessEntityID'], how='outer', suffixes=('', '_emailAddress'))
person = pd.merge(person, personPhone, on=['BusinessEntityID'], how='outer', suffixes=('', '_personPhone'))
person = pd.merge(person, phoneNumberType, on=['PhoneNumberTypeID'], how='outer', suffixes=('', '_phoneNumberType'))
person = pd.merge(person, businessEntityContact, on=['BusinessEntityID'], how='outer', suffixes=('', '_businessEntityContact'))
person = pd.merge(person, contactType, on=['ContactTypeID'], how='outer', suffixes=('', '_contactType'))

#Get most recent ModifiedDate
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_businessEntity'] if pd.to_datetime(row['ModifiedDate_businessEntity']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_password'] if pd.to_datetime(row['ModifiedDate_password']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_emailAddress'] if pd.to_datetime(row['ModifiedDate_emailAddress']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_personPhone'] if pd.to_datetime(row['ModifiedDate_personPhone']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_phoneNumberType'] if pd.to_datetime(row['ModifiedDate_phoneNumberType']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_businessEntityContact'] if pd.to_datetime(row['ModifiedDate_businessEntityContact']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_contactType'] if pd.to_datetime(row['ModifiedDate_contactType']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
person.drop(columns=['ModifiedDate_businessEntity'], inplace=True)
person.drop(columns=['ModifiedDate_password'], inplace=True)
person.drop(columns=['ModifiedDate_emailAddress'], inplace=True)
person.drop(columns=['ModifiedDate_personPhone'], inplace=True)
person.drop(columns=['ModifiedDate_phoneNumberType'], inplace=True)
person.drop(columns=['ModifiedDate_businessEntityContact'], inplace=True)
person.drop(columns=['ModifiedDate_contactType'], inplace=True)

person

KeyboardInterrupt: 

### Product

In [ ]:
#Collecting tables
product_adventureworks = pd.read_sql('SELECT * FROM "Production.Product"', adventureworks_sqlite_conn)
productModel = pd.read_sql('SELECT * FROM "Production.ProductModel"', adventureworks_sqlite_conn)
productSubcategory = pd.read_sql('SELECT * FROM "Production.ProductSubcategory"', adventureworks_sqlite_conn)
productCategory = pd.read_sql('SELECT * FROM "Production.ProductCategory"', adventureworks_sqlite_conn)

product_northwind = pd.read_sql('SELECT * FROM "Products"', northwind_sqlite_conn)
categories = pd.read_sql('SELECT * FROM "Categories"', northwind_sqlite_conn)
suppliers = pd.read_sql('SELECT * FROM "Suppliers"', northwind_sqlite_conn)

product_aenc = pd.read_sql('SELECT * FROM "product"', aenc_sqlite_conn)

#Merges
product_adventureworks = pd.merge(product_adventureworks, productModel, on=['ProductModelID'], how='outer', suffixes=('', '_productModel'))
product_adventureworks = pd.merge(product_adventureworks, productSubcategory, on=['ProductSubcategoryID'], how='outer', suffixes=('', '_productSubcategory'))
product_adventureworks = pd.merge(product_adventureworks, productCategory, on=['ProductCategoryID'], how='outer', suffixes=('', '_productCategory'))

product_northwind = pd.merge(product_northwind, categories, on=['CategoryID'], how='outer', suffixes=('', '_Category'))
product_northwind = pd.merge(product_northwind, suppliers, on=['SupplierID'], how='outer', suffixes=('', '_Supplier'))

product = pd.concat([product_adventureworks, product_northwind, product_aenc])

#Get most recent ModifiedDate
product['ModifiedDate'] = product.apply(lambda row: row['ModifiedDate_productModel'] if pd.to_datetime(row['ModifiedDate_productModel']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = product.apply(lambda row: row['ModifiedDate_productCategory'] if pd.to_datetime(row['ModifiedDate_productCategory']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = product.apply(lambda row: row['ModifiedDate_productSubcategory'] if pd.to_datetime(row['ModifiedDate_productSubcategory']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
product.drop(columns=['ModifiedDate_productModel'], inplace=True)
product.drop(columns=['ModifiedDate_productCategory'], inplace=True)
product.drop(columns=['ModifiedDate_productSubcategory'], inplace=True)

product

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,HomePage,id,name,description,prod_size,color,quantity,unit_price,picture_name,Category
0,771.0,"Mountain-100 Silver, 38",BK-M82S-38,1.0,1.0,Silver,100.0,75.0,1912.1544,3399.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,772.0,"Mountain-100 Silver, 42",BK-M82S-42,1.0,1.0,Silver,100.0,75.0,1912.1544,3399.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,773.0,"Mountain-100 Silver, 44",BK-M82S-44,1.0,1.0,Silver,100.0,75.0,1912.1544,3399.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,774.0,"Mountain-100 Silver, 48",BK-M82S-48,1.0,1.0,Silver,100.0,75.0,1912.1544,3399.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,775.0,"Mountain-100 Black, 38",BK-M82B-38,1.0,1.0,Black,100.0,75.0,1898.0944,3374.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,500,Visor,Cloth Visor,One size fits all,White,36,7,visorw.bmp,Accessories
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,501,Visor,Plastic Visor,One size fits all,Black,28,7,visorb.bmp,Accessories
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,600,Sweatshirt,Hooded Sweatshirt,Large,Green,39,24,sshirtg.bmp,Clothes
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,601,Sweatshirt,Zipped Sweatshirt,Large,Blue,32,24,sshirtb.bmp,Clothes


### Employees

In [127]:
#Collecting tables
employee_adventureworks = pd.read_sql('SELECT * FROM "HumanResources.Employee"', adventureworks_sqlite_conn)

employee_northwind = pd.read_sql('SELECT * FROM "Employees"', northwind_sqlite_conn)
employee_territories = pd.read_sql('SELECT * FROM "EmployeeTerritories"', northwind_sqlite_conn)

employee_aenc = pd.read_sql('SELECT * FROM "employee"', aenc_sqlite_conn)

#Merges
employee_northwind = pd.merge(employee_northwind, employee_territories, on=['EmployeeID'], how='outer', suffixes=('', '_employee_territories'))

employee = pd.concat([employee_adventureworks, employee_northwind, employee_aenc])

employee

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,...,status,ss_number,salary,start_date,termination_date,birth_date,bene_health_ins,bene_life_ins,bene_day_care,sex
0,1.0,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,245797967,adventure-works\terri0,b'X',1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,509647174,adventure-works\roberto0,b'Z\xc0',2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,112457891,adventure-works\rob0,b'Z\xd6',3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,695256908,adventure-works\gail0,b'Z\xda',3.0,Design Engineer,1952-09-27,M,F,2008-01-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,023857807,29384,15-Jun-1997 12:00:00 AM,None,12-Sep-1969 12:00:00 AM,Y,Y,N,F
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,023850840,24903,28-Aug-1997 12:00:00 AM,None,18-Jan-1974 12:00:00 AM,Y,Y,N,M
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,038957036,45829,13-Sep-1997 12:00:00 AM,None,31-Oct-1970 12:00:00 AM,Y,Y,N,F
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,037534650,34889,23-Dec-1997 12:00:00 AM,None,19-Jun-1966 12:00:00 AM,Y,Y,N,M


### Customer

In [131]:
#Collecting tables
customer_adventureworks = pd.read_sql('SELECT * FROM "Sales.Customer"', adventureworks_sqlite_conn)

customer_northwind = pd.read_sql('SELECT * FROM "Customers"', northwind_sqlite_conn)

customer_aenc = pd.read_sql('SELECT * FROM "customer"', aenc_sqlite_conn)

#Merges
customer = pd.concat([customer_adventureworks, customer_northwind, customer_aenc])

customer

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate,CompanyName,ContactName,ContactTitle,...,Fax,id,fname,lname,address,city,state,zip,phone,company_name
0,1,NaN,934.0,1.0,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1028.0,1.0,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,642.0,4.0,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,932.0,4.0,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,1026.0,4.0,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,552,Janice,O'Toole,654 West Hill,Nashville,TN,37320,6155553689,Greensleeves
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,553,Stevie,Nickolas,77 Recordings Circle,Tacoma,WA,96521,5095551695,It's a Hit!
123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,555,Philipe,Fernandez,99 Main Street,Los Angeles,CA,90205,2135554457,Quaker Fashions
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,661,Jennifer,Stutzman,3 Back Pages Lane,Missola,IL,60505,7085556857,Stutzman Advertising


### Connecties sluiten

In [77]:
northwind_sqlite_conn.close()
adventureworks_sqlite_conn.close()
aenc_sqlite_conn.close()